In [1]:
# Install kerastuner for hyperpaparameter tuning
pip install keras-tuner

     |████████████████████████████████| 97 kB 3.1 MB/s 


In [2]:
# Import all the libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
## Load the dataset
fashion_mnsit = keras.datasets.fashion_mnist

In [4]:
fashion_mnsit

<module 'keras.api._v2.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/keras/api/_v2/keras/datasets/fashion_mnist/__init__.py'>

In [5]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnsit.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
## Coverting all the images to the grayscale images
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
test_images[0].shape

(28, 28)

In [9]:
## Reshaping the images (number of images, size, Gray scale)
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [21]:
## Now create our hyperparameter tuning function to select the best parameters

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [22]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [23]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [24]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 53s]
val_accuracy: 0.9144999980926514

Best val_accuracy So Far: 0.9144999980926514
Total elapsed time: 00h 04m 32s
INFO:tensorflow:Oracle triggered exit


In [25]:
model=tuner_search.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,719,434
Trainable params: 2,719,434
Non-trainable params: 0
_________________________________________________________________


In [28]:
# fitting the model
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.1287 - accuracy: 0.9513 - val_loss: 0.2749 - val_accuracy: 0.9118
Epoch 5/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0882 - accuracy: 0.9679 - val_loss: 0.3200 - val_accuracy: 0.9115
Epoch 6/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0621 - accuracy: 0.9776 - val_loss: 0.4092 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0440 - accuracy: 0.9839 - val_loss: 0.4405 - val_accuracy: 0.9087
Epoch 8/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0367 - accuracy: 0.9869 - val_loss: 0.4934 - val_accuracy: 0.9090
Epoch 9/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0304 - accuracy: 0.9896 - val_loss: 0.4754 - val_accuracy: 0.9107
Epoch 10/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0228 - accuracy: 0.9921 - val_loss: 0.5110 

In [34]:
# fitting model for test dataset
model.predict(test_images)

In [36]:
loss, accuracy = model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.5338 - accuracy: 0.9119


In [37]:
# Acquired test accuracy of 91.1% 